# Preparación de Datos para Looker Studio

**Curso:** CD2001B - Diagnóstico para Líneas de Acción  
**Semana 4:** Visualización de Datos y Dashboards  
**Objetivo:** Preparar y exportar datos de tu ONG en formato óptimo para Looker Studio

---

## 📋 Contenido

1. Limpieza y transformación de datos
2. Creación de variables calculadas
3. Agregaciones para KPIs
4. Formato óptimo para Looker Studio
5. Exportación (CSV y Google Sheets)

## 1. Cargar y Limpiar Datos

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# Cargar tus datos de la ONG
# df = pd.read_csv('ruta_a_tus_datos.csv')

# Datos de ejemplo
np.random.seed(42)
n = 500

df = pd.DataFrame({
    'fecha': pd.date_range(start='2024-01-01', periods=n, freq='D')[:n],
    'beneficiario_id': np.arange(1, n + 1),
    'edad': np.random.choice(range(5, 65), n),
    'genero': np.random.choice(['F', 'M'], n),
    'municipio': np.random.choice(['Puebla', 'Cholula', 'Atlixco', 'Tehuacán'], n),
    'tipo_servicio': np.random.choice(['Servicio A', 'Servicio B', 'Servicio C'], n),
    'satisfaccion': np.random.randint(1, 11, n),
    'tiempo_atencion_min': np.random.randint(10, 120, n),
    'costo': np.random.uniform(0, 500, n)
})

print(f"✅ Datos cargados: {len(df)} registros")
df.head()

In [ ]:
# Limpieza básica
print("LIMPIEZA DE DATOS")
print("="*60)

# 1. Eliminar duplicados
duplicados_antes = len(df)
df = df.drop_duplicates()
print(f"Duplicados eliminados: {duplicados_antes - len(df)}")

# 2. Manejar valores faltantes
print(f"\nValores faltantes:")
print(df.isnull().sum())

# 3. Estandarizar formatos de texto
text_cols = df.select_dtypes(include=['object']).columns
for col in text_cols:
    if col != 'fecha':
        df[col] = df[col].str.strip().str.title()

print("\n✅ Limpieza completada")

## 2. Crear Variables Calculadas

Looker Studio se beneficia de tener variables pre-calculadas.

In [ ]:
# Convertir fecha a datetime si no lo es
df['fecha'] = pd.to_datetime(df['fecha'])

# Variables de tiempo
df['año'] = df['fecha'].dt.year
df['mes'] = df['fecha'].dt.month
df['mes_nombre'] = df['fecha'].dt.strftime('%B')
df['trimestre'] = df['fecha'].dt.quarter
df['semana'] = df['fecha'].dt.isocalendar().week
df['dia_semana'] = df['fecha'].dt.day_name()

# Categorías útiles
df['grupo_edad'] = pd.cut(df['edad'], 
                           bins=[0, 12, 18, 30, 50, 100],
                           labels=['Niños', 'Adolescentes', 'Jóvenes', 'Adultos', 'Adultos Mayores'])

df['nivel_satisfaccion'] = pd.cut(df['satisfaccion'],
                                   bins=[0, 4, 7, 10],
                                   labels=['Baja', 'Media', 'Alta'])

df['tiempo_categoria'] = pd.cut(df['tiempo_atencion_min'],
                                 bins=[0, 30, 60, 120],
                                 labels=['Rápido', 'Normal', 'Lento'])

# Variable binaria para KPI
df['satisfecho'] = (df['satisfaccion'] >= 8).astype(int)
df['atencion_rapida'] = (df['tiempo_atencion_min'] <= 30).astype(int)

print("✅ Variables calculadas creadas")
print(f"\nNuevas columnas: {df.columns.tolist()[-10:]}")

## 3. Tabla de KPIs Agregados

Crea una tabla resumen para mostrar KPIs en tu dashboard.

In [ ]:
# KPIs por mes
kpis_mensuales = df.groupby(['año', 'mes', 'mes_nombre']).agg({
    'beneficiario_id': 'count',  # Total de atenciones
    'satisfaccion': 'mean',  # Satisfacción promedio
    'satisfecho': 'mean',  # % satisfechos
    'tiempo_atencion_min': 'mean',  # Tiempo promedio
    'atencion_rapida': 'mean',  # % atención rápida
    'costo': 'sum'  # Costo total
}).reset_index()

kpis_mensuales.columns = ['Año', 'Mes_Num', 'Mes', 'Total_Atenciones', 
                          'Satisfaccion_Promedio', 'Porcentaje_Satisfechos',
                          'Tiempo_Promedio_Min', 'Porcentaje_Atencion_Rapida', 'Costo_Total']

# Convertir porcentajes
kpis_mensuales['Porcentaje_Satisfechos'] = kpis_mensuales['Porcentaje_Satisfechos'] * 100
kpis_mensuales['Porcentaje_Atencion_Rapida'] = kpis_mensuales['Porcentaje_Atencion_Rapida'] * 100

print("KPIs MENSUALES")
print("="*60)
print(kpis_mensuales.head(10))

In [ ]:
# KPIs por ubicación
kpis_ubicacion = df.groupby('municipio').agg({
    'beneficiario_id': 'count',
    'satisfaccion': 'mean',
    'tiempo_atencion_min': 'mean',
    'costo': 'mean'
}).reset_index()

kpis_ubicacion.columns = ['Municipio', 'Total_Atenciones', 'Satisfaccion_Promedio',
                          'Tiempo_Promedio_Min', 'Costo_Promedio']

print("\nKPIs POR UBICACIÓN")
print("="*60)
print(kpis_ubicacion)

## 4. Formato Óptimo para Looker Studio

### Mejores prácticas:
- ✅ Nombres de columnas claros (sin caracteres especiales)
- ✅ Fechas en formato ISO (YYYY-MM-DD)
- ✅ Números sin formato de miles (usa punto decimal)
- ✅ Categorías como texto, no números codificados
- ✅ Una fila por observación (formato largo)

In [ ]:
# Preparar dataset principal para Looker
df_looker = df.copy()

# Renombrar columnas para Looker (sin espacios, claras)
df_looker = df_looker.rename(columns={
    'beneficiario_id': 'ID_Beneficiario',
    'fecha': 'Fecha',
    'edad': 'Edad',
    'genero': 'Genero',
    'municipio': 'Municipio',
    'tipo_servicio': 'Tipo_Servicio',
    'satisfaccion': 'Satisfaccion',
    'tiempo_atencion_min': 'Tiempo_Atencion_Min',
    'costo': 'Costo',
    'grupo_edad': 'Grupo_Edad',
    'nivel_satisfaccion': 'Nivel_Satisfaccion',
    'satisfecho': 'Satisfecho_Flag',
    'atencion_rapida': 'Atencion_Rapida_Flag'
})

# Convertir categorías a string
for col in ['Grupo_Edad', 'Nivel_Satisfaccion', 'tiempo_categoria']:
    if col in df_looker.columns:
        df_looker[col] = df_looker[col].astype(str)

# Formato de fecha
df_looker['Fecha'] = df_looker['Fecha'].dt.strftime('%Y-%m-%d')

print("✅ Datos formateados para Looker Studio")
print(f"Columnas finales: {df_looker.columns.tolist()}")

## 5. Exportación

### Opción A: Exportar a CSV

In [ ]:
# Exportar dataset principal
df_looker.to_csv('../datos/datos_looker_principal.csv', index=False, encoding='utf-8-sig')
print("✅ Exportado: datos_looker_principal.csv")

# Exportar tabla de KPIs mensuales
kpis_mensuales.to_csv('../datos/kpis_mensuales.csv', index=False, encoding='utf-8-sig')
print("✅ Exportado: kpis_mensuales.csv")

# Exportar KPIs por ubicación
kpis_ubicacion.to_csv('../datos/kpis_ubicacion.csv', index=False, encoding='utf-8-sig')
print("✅ Exportado: kpis_ubicacion.csv")

### Opción B: Conectar con Google Sheets

**Pasos para usar Google Sheets con Looker Studio:**

1. **Subir CSV a Google Sheets:**
   - Abre Google Sheets
   - File → Import → Upload → Selecciona tu CSV
   - Elige "Replace spreadsheet" o "Insert new sheet"

2. **O usar gspread (Python):**

In [ ]:
# Código de ejemplo (requiere configuración de credenciales)
# Descomentar y configurar si quieres usar esta opción

# !pip install gspread oauth2client

# import gspread
# from oauth2client.service_account import ServiceAccountCredentials

# scope = ['https://spreadsheets.google.com/feeds',
#          'https://www.googleapis.com/auth/drive']

# creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
# client = gspread.authorize(creds)

# spreadsheet = client.create('Datos ONG para Looker')
# worksheet = spreadsheet.get_worksheet(0)

# # Subir datos
# worksheet.update([df_looker.columns.values.tolist()] + df_looker.values.tolist())

# print(f"✅ Datos subidos a Google Sheets: {spreadsheet.url}")

print("\n💡 TIP: Para conectar con Google Sheets, sigue el tutorial en la guía de Looker Studio")

## 6. Diccionario de Datos

Crea un diccionario de datos para documentar tus variables.

In [ ]:
diccionario = pd.DataFrame({
    'Variable': df_looker.columns,
    'Tipo': [str(df_looker[col].dtype) for col in df_looker.columns],
    'Descripcion': [
        'Identificador único del beneficiario',
        'Fecha de la atención (YYYY-MM-DD)',
        'Edad del beneficiario en años',
        'Género: F (Femenino) o M (Masculino)',
        'Municipio de procedencia',
        'Tipo de servicio recibido',
        'Nivel de satisfacción (1-10)',
        'Tiempo de atención en minutos',
        'Costo del servicio en pesos',
        'Año de la atención',
        'Mes (número)',
        'Mes (nombre)',
        'Trimestre (1-4)',
        'Semana del año',
        'Día de la semana',
        'Grupo etario: Niños/Adolescentes/Jóvenes/Adultos/Adultos Mayores',
        'Nivel de satisfacción: Baja/Media/Alta',
        'Categoría de tiempo: Rápido/Normal/Lento',
        'Flag: 1 si satisfacción >= 8, 0 si no',
        'Flag: 1 si tiempo <= 30 min, 0 si no'
    ][:len(df_looker.columns)]
})

diccionario.to_csv('../datos/diccionario_datos.csv', index=False, encoding='utf-8-sig')
print("DICCIONARIO DE DATOS")
print("="*60)
print(diccionario.to_string(index=False))
print("\n✅ Exportado: diccionario_datos.csv")

## ✅ Checklist Final

Antes de crear tu dashboard en Looker Studio:

- [ ] Datos limpios (sin duplicados, valores faltantes manejados)
- [ ] Variables de tiempo creadas (año, mes, trimestre)
- [ ] Categorías útiles creadas (grupos de edad, niveles)
- [ ] Flags binarios para KPIs creados
- [ ] Tabla de KPIs agregados preparada
- [ ] Nombres de columnas sin espacios ni caracteres especiales
- [ ] Fechas en formato ISO
- [ ] Datos exportados a CSV o Google Sheets
- [ ] Diccionario de datos documentado

---

## 📚 Próximos Pasos

1. **Sube tus archivos CSV a Google Sheets** (o usa directamente CSV)
2. **Abre Looker Studio**: https://lookerstudio.google.com/
3. **Crea un nuevo reporte** y conecta tu fuente de datos
4. **Sigue el notebook** `02_visualizacion_python.ipynb` para ideas de gráficos

---

**💡 RECURSOS:**
- Guía de Looker Studio: `/Semana4/guias/guia_looker_studio_basico.md`
- Ejemplo de dashboard: `/Semana4/ejemplos/dashboard_ejemplo_ong.pdf`
- Plantilla dashboard: `/Semana4/plantillas/plantilla_dashboard_ong.md`
